# [Классификация текстов обзоров фильмов с Keras и TensorFlow Hub](https://www.tensorflow.org/tutorials/keras/text_classification_with_hub)



В этом уроке мы будем классифицировать обзоры фильмов как *позитивные* или *негативные* , используя текст обзора. Это пример бинарной или двуклассовой классификации, важный и широко применяемый тип задач машинного обучения.

Учебное руководство демонстрирует применение переносимого обучения с использованием TensorFlow Hub и Keras.

Мы будем использовать [набор данных IMDB](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/imdb) содержащий тексты 50 000 обзоров фильмов из [базы данных фильмов в Интернете](https://www.imdb.com/). Они разделены на 25 000 обзоров для обучения и 25 000 обзоров для проверки моделей. Обучающая и тестовая выборка сбалансированы , т.е. содержат одинаковое количество *позитивных* и *негативных обзоров*.

Здесь мы будем использовать `tf.keras` , высокоуровневый API для построения и обучения моделей в TensorFlow и TensorFlow Hub , библиотека и платформа для переноса обучения. Для более продвинутых сообщений по классификации текстов с использованием `tf.keras`.


In [ ]:
try:
  # Colab only
  %tensorflow_version 2.x
except Exception:
    pass

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np

import tensorflow as tf

import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

## Загружаем датасет



Датасет IMDB доступен в [датасетах TensorFlow](https://github.com/tensorflow/datasets/tree/master/tensorflow_datasets). Следующий код скачивает датасет IMDB на ваш компьютер (или в среду выполнения Colab):


In [ ]:
(train_data, validation_data), info = tfds.load(
    name="imdb_reviews", 
    # Возращает тренировочный/тестовый наборы данных как кортеж.
    split = (tfds.Split.TRAIN, tfds.Split.TEST),
    # Возрашать (примеры, лейблы) парами из набора данных (вместо словаря).
    as_supervised=True,
    # Также попросим вернуть структуру "информация".
    with_info=True)

## Исследование данных



Давайте уделим немного времени, чтобы разобраться в формате данных. Каждый пример представляет собой предложение являющееся обзором фильма и соответствующую метку. Предложение никак не предобработано. Метка является целым числом, 0 или 1, где 0 - это отрицательный отзыв, а 1 - положительный.

Выведем первые 10 примеров.

In [ ]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
train_examples_batch

## Построить модель


В этом примере, входные данные состоят из предложений. Метки которые нужно предсказать являются 0 либо 1.

Одним из способов представления текста является преобразование предложений в векторные представления слов. Мы можем использовать предварительно обученное векторное представление текста в качестве первого слоя. Это имеет три преимущества:

* нам не нужно беспокоиться о препроцессинге текста,
* мы можем извлечь выгоду из переноса обучения,
* векторное представление фиксированного размера, поэтому его проще обрабатывать.

## TensorFlow Hub

[**TensorFlow Hub**](https://www.tensorflow.org/hub?hl=ru)  —  это платформа, где можно публиковать, изучать и использовать модули машинного обучения, написанные в TensorFlow. Под модулем мы подразумеваем самодостаточную и обособленную часть графа TensorFlow (с обученными весами), которая может быть использована в других задачах. С помощью модуля разработчик cможет обучить модель на меньшем датасете, улучшить способность обобщать или просто увеличить скорость обучения.

Для этого примера мы используем **предобученную модель векторного представления текста** из TensorFlow Hub называемую [`google/tf2-preview/gnews-swivel-20dim/1`](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1).

Есть еще три другие предварительно обученных модели подходящие для этого руководства:

* [`google/tf2-preview/gnews-swivel-20dim-with-oov/1`](https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim-with-oov/1) - аналогичная `google/tf2-preview/gnews-swivel-20dim/1`, но с 2.5% словаря конвертированного в OOV buckets. Это может помочь, если словарь задачи и словарь модели не полностью совпадают.
* [`google/tf2-preview/nnlm-en-dim50/1`](https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1) - Намного большая модель с размером словаря ~1M и размерностью вложения 50.
* [`google/tf2-preview/nnlm-en-dim128/1`](https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1) - Еще большая модель с размером словаря ~1M и размерностью вложения 128.

Давайте сначала создадим слой Keras, который использует модель TensorFlow Hub для векторного представления предложений, и опробуем его на нескольких входных примерах. Обратите внимание, что независимо от длины входного текста, размерность векторного представления будет следующей: `(num_examples, embedding_dimension)`.







In [ ]:
embed = hub.load("https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1")
embeddings = embed(["cat is on the mat", "dog is in the fog", "cat is on the mat dog is in the fog"])
embeddings

## Вход `gnews-swivel-20dim`
В качестве входных данных модуль принимает пакет предложений в одномерном тензоре строк.

## Предварительная обработка
Модуль предварительно обрабатывает свой ввод путем разбиения на пробелы .

## Словарь
Словарь содержит 20 000 токенов и 1 из словарного запаса для неизвестных токенов.

## Векторное представление предложений

Векторное представление слов комбинируется в векторное представление слов используя `sqrtn` - взвешенная сумма, деленная на квадратный корень из суммы квадратов весов (подробнее см. [tf.nn.embedding_lookup_sparse](https://www.tensorflow.org/api_docs/python/tf/nn/embedding_lookup_sparse)).



In [ ]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])

Давайте построим полную модель:

In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Для построения классификатора зададим слои последовательно:

1. Первый слой это слой TensorFlow Hub. Этот слой использует предобученную Saved Model, отображающую предложения в векторные представления. Предобученная модель векторного представления слов которую мы используем (`google/tf2-preview/gnews-swivel-20dim/1`) разбивает предложение на токены, встраивает каждый токен и затем объединяет вложения. В результате получаются размерности: `(num_examples, embedding_dimension)`.
1. Получившийся в результате вектор фиксированной длины пропускается сквозь полносвязный (Dense) слой состоящий из 16 скрытых нейронов.
1. Последний слой плотно связан с единственным выходным нейроном. С использованием функции активации сигмоида, значение получается между 0 и 1, представляя вероятность или уровень доверия.

Давайте скомпилируем модель.


## Функция потерь и оптимизатор



Для модели нам необходимо указать функцию потерь и оптимизатор для обучения. Поскольку мы решаем задачу бинарной классификации и на выходе модели будут вероятности (слой из единственного элемента с сигмоидой в качестве функции активации), то мы воспользуемся функцией потерь `binary_crossentropy`.

Это не единственный выбор для функции потерь: Вы можете, например, выбрать `mean_squared_error`. Но обычно `binary_crossentropy` лучше справляется с вероятностями - она измеряет "дистанцию" между распределениями вероятностей, или, как в нашем случае, между истинным распределением и предсказаниями.

Настроим модель с использованием оптимизатора и функции потерь:

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## Обучите модель

Обучите модель за 20 эпох в мини-батчах по 512 образцов. Это 20 итераций по всем образцам данных в тензорах `x_train` и `y_train`. Во время обучениЯ следите за функцией потерь и точностью на 10 000 примеров из проверочного набора данных:

In [ ]:
history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

## Оценка модели



Давайте посмотрим как работает модель. Она будет возвращать два значения. Потери (число, показывающее нашу ошибку, меньшие значения - лучше) и точность (accuracy).

In [ ]:
results = model.evaluate(validation_data.batch(512), verbose=2)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

Этот довольно наивный подход достиг точности около 87%. С более продвинутыми методами модель бы приблизилась к 95%.

## Алгоритм Swivel и его приложения



Алгоритмы машинного обучения обычно ожидают ввода чисел. Когда ученый хочет использовать текст для создания модели машинного обучения, он должен сначала найти способ представить свой текст как вектор чисел. Эти векторы называются word embeddings ([(векторное) представление слова](http://www.machinelearning.ru/wiki/index.php?title=%D0%A1%D0%BB%D0%BE%D0%B2%D0%B0%D1%80%D1%8C_%D1%82%D0%B5%D1%80%D0%BC%D0%B8%D0%BD%D0%BE%D0%B2_%D0%BC%D0%B0%D1%88%D0%B8%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE_%D0%BE%D0%B1%D1%83%D1%87%D0%B5%D0%BD%D0%B8%D1%8F), погружение слова в линейное векторное пространство). **Алгоритм Swivel** - это представление слов на основе частоты, использующее матрицу совместного вхождения. **Идея** здесь заключается в том, что слова, имеющие сходное значение, как правило, встречаются вместе в текстовом корпусе. В результате слова, которые имеют схожие значения, будут иметь векторные представления, которые ближе, чем слова несвязанных слов.

* Оригинальная статья [здесь](https://arxiv.org/abs/1602.02215).

* См. презентацию [здесь](https://zurk.github.io/moscow-python-06-2018/). 

* См. доклад [здесь](https://youtu.be/g3vsayMXjjE)

* Github [Swivel in Tensorflow](https://github.com/tensorflow/models/tree/master/research/swivel)


